<a href="https://colab.research.google.com/github/ghlai9665/transformer-implementation/blob/main/Transformer_Implementation_PyTorch_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Dependencies


In [ ]:
# Helper functions for examining GPU memory
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
print(GPUs)
gpu = GPUs[0]
print(gpu)
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

import gc
def print_allocations(mode='get_number'):
  # if you want to print out the actual tensors
  if mode == 'get_tensors':
    for obj in gc.get_objects():
        try:
            
            if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                print(type(obj), obj.size())
        except:
            pass
  else:
    count = 0
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
              count += 1
        except:
            pass
    print(count)

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=f1a2d626cf5f593650281fbd7e51345465dc99c3a42ed66ecb478c2acf617413
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 25.6 GB  | Proc size: 1.7 GB
GPU RAM Free: 16130MB | Used: 0MB | Util   0% | Total 16130MB


In [ ]:
!pip uninstall torchtext torch
!pip install --pre torch torchtext -f https://download.pytorch.org/whl/nightly/cu101/torch_nightly.html
!pip install numpy matplotlib spacy seaborn

In [ ]:
import datetime
import pytz
tz = pytz.timezone('Asia/Chongqing')

from google.colab import files, drive
drive.mount('/content/gdrive')
model_name = 'iwslt_checkpoint.pth'
path = F"/content/gdrive/My Drive/transformer/{model_name}" 

Mounted at /content/gdrive


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn
import math
import time
seaborn.set_context(context="talk")
%matplotlib inline

In [ ]:
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]; available_gpus
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print(ram_gb)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'); device

27.393773568


device(type='cuda')

In [ ]:
!pip show torchtext

Name: torchtext
Version: 0.9.0.dev20210210
Summary: Text utilities and datasets for PyTorch
Home-page: https://github.com/pytorch/text
Author: PyTorch core devs and James Bradbury
Author-email: jekbradbury@gmail.com
License: BSD
Location: /usr/local/lib/python3.6/dist-packages
Requires: requests, numpy, torch, tqdm
Required-by: 


# Load Data

In [ ]:
!python -m spacy download en
!python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9MB 17.8MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907057 sha256=e6c2b800f66e414302dff75d33c6c4d492e87312cb8411b414b0419778d1d611
  Stored in directory: /tmp/pip-ephem-wheel-cache-ywv0hqvu/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [ ]:
from torchtext.experimental.datasets import IWSLT
from torchtext.data.utils import get_tokenizer

# tokenize the dataset (pairs of sentence strings -> pairs of arrays of indices)
src_tokenizer = get_tokenizer("spacy", language='de')
tgt_tokenizer = get_tokenizer("spacy", language='en')
train_dataset, valid_dataset, test_dataset = IWSLT(tokenizer=(src_tokenizer, tgt_tokenizer))
# vocab allows us see which index maps to which word
de_vocab, en_vocab = train_dataset.get_vocab()

100%|██████████| 196884/196884 [00:20<00:00, 9464.47lines/s]


In [ ]:
class Batch:
    def __init__(self, src, trg=None, pad_value=1, d_model=128):
        src_vocab_size, trg_vocab_size = len(de_vocab.itos), len(en_vocab.itos)
        # input src & trg are shape (batch_size, sentence_len) and embedded into (batch_size, sentence_len, d_model)
        # finally transposed into (sentence_len, batch_size, d_model)
        assert src.shape[0] == trg.shape[0], "src and trg should have the same batch size!"

        batch_size, src_sentence_len = src.shape

        self.src_padding_mask = get_padding_mask(src)
        self.src = embed(src, src_vocab_size, d_model=d_model).transpose(0, 1)
        
        if trg != None:
            # given src & trg_x...
            self.trg_padding_mask = get_padding_mask(trg[:,:-1])
            self.trg_x = embed(trg, trg_vocab_size, d_model=d_model)[:, :-1, :].transpose(0, 1)
            trg_sentence_len = self.trg_x.shape[0]
            self.trg_attn_mask = generate_square_subsequent_mask(trg_sentence_len)
            # ...we try to predict trg_y, which has ntokens words (i.e. we make ntokens predictions)
            self.trg_y = trg[:, 1:].transpose(0,1)
            self.ntokens = int((self.trg_y != pad_value).sum())

In [ ]:
# this tells dataloader how you want your batch to look like 
# input is a list of tensors of size batch_size (dataloader just feeds you a mini-batch of batch_size at a time and you can process it),
# where each tensor is [src, trg]
# output is whatever you want in train_epoch
def collate_batch(batch_data, pad_idx=1, d_model = 128):
    max_src_len = max([len(sentence_pair[0]) for sentence_pair in batch_data])
    max_trg_len = max([len(sentence_pair[1]) for sentence_pair in batch_data])
    # initialize the padding in the shape of the result src/trg we want
    res_src = torch.zeros(len(batch_data), max_src_len).long() + pad_idx
    res_trg = torch.zeros(len(batch_data), max_trg_len).long() + pad_idx
    # layer the actual sentence on top of the padding
    for i, sentence_pair in enumerate(batch_data):
        src_sentence, trg_sentence = sentence_pair
        res_src[i, :len(src_sentence):], res_trg[i, :len(trg_sentence):] = src_sentence.long(), trg_sentence.long() # the first part of sentence are filled with words, the rest are pads
        
    return Batch(res_src, res_trg, pad_value=de_vocab.stoi['<pad>'], d_model=d_model)

In [ ]:
# divide the data into batches, using Dataloader
sorted_train_dataset = sorted(train_dataset, key=lambda x: (len(x[0]), len(x[1])), reverse=True)

In [ ]:
def sentences_not_too_long(sentence_pair):
  src_sentence, trg_sentence = sentence_pair
  return len(src_sentence) <= 50 and len(trg_sentence) <= 50

filtered_train_dataset = list(filter(sentences_not_too_long, sorted_train_dataset))
filtered_valid_dataset = list(filter(sentences_not_too_long, valid_dataset))

In [ ]:
print(len(valid_dataset))
print(len(filtered_valid_dataset))

993
993


In [ ]:
print(len(filtered_train_dataset) / len(sorted_train_dataset))

0.9589809227768635


In [ ]:
d_model = 128

In [ ]:
# the output from a dataloader must be of shape u
train_dataloader = DataLoader(filtered_train_dataset, batch_size=16, shuffle=True, collate_fn=lambda b: collate_batch(b, d_model=d_model))

In [ ]:
valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=True, collate_fn=lambda b: collate_batch(b, d_model=d_model))

In [ ]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

# input shape (N, S), you need to get src_padding_mask and trg_padding_mask
def get_padding_mask(batch, pad_index = 1):
    # shape (N, S)
    padding_mask = (batch == pad_index)
    return padding_mask

# Explore Data

In [ ]:
def print_top(n_rows, mode='words'):
  if mode == 'words':
    for i in range(n_rows):
      de_sentence = [de_vocab.itos[index] for index in train_dataset[i][0]]
      en_sentence = [en_vocab.itos[index] for index in train_dataset[i][1]]
      print((de_sentence, en_sentence))
  elif mode == 'indices':
    for i in range(n_rows):
      print(train_dataset[i])

print_top(100, 'words')
print_top(10, 'indices')

In [ ]:
print(de_vocab.stoi['<pad>'])
print(en_vocab.stoi['<pad>'])

1
1


In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [ ]:
def embed(x, vocab_size, d_model=512):
    # two embeddings (1) each token's numeral value is mapped to a embedding vector, index (scalar) -> embedding vector (size of d_model) 
    # (2) positional embedding is applied
    number_to_embedding = nn.Embedding(vocab_size, d_model)
    pos_embedding = PositionalEncoding(d_model)
    x = number_to_embedding(x) * math.sqrt(d_model)
    x = pos_embedding(x)
    return x

In [ ]:
for i, batch in enumerate(train_dataloader):
    if i > 10:
      break
    print("## batch.trg_x: ", batch.trg_x.shape)

## batch.trg_x:  torch.Size([69, 16, 128])
## batch.trg_x:  torch.Size([70, 16, 128])
## batch.trg_x:  torch.Size([75, 16, 128])
## batch.trg_x:  torch.Size([42, 16, 128])
## batch.trg_x:  torch.Size([49, 16, 128])
## batch.trg_x:  torch.Size([61, 16, 128])
## batch.trg_x:  torch.Size([76, 16, 128])
## batch.trg_x:  torch.Size([48, 16, 128])
## batch.trg_x:  torch.Size([223, 16, 128])
## batch.trg_x:  torch.Size([58, 16, 128])
## batch.trg_x:  torch.Size([56, 16, 128])


In [ ]:
for i, batch in enumerate(valid_dataloader):
    if i > 4:
      break
    print("## batch.trg_x: ", batch.trg_x.shape)

## batch.trg_x:  torch.Size([59, 16, 128])
## batch.trg_x:  torch.Size([38, 16, 128])
## batch.trg_x:  torch.Size([42, 16, 128])
## batch.trg_x:  torch.Size([51, 16, 128])
## batch.trg_x:  torch.Size([80, 16, 128])


# Define Functions for Training

In [ ]:
class Generator(nn.Module):
  def __init__(self, d_model, vocab):
    super().__init__()
    self.proj = nn.Linear(d_model, vocab) # define the linear projection, which takes d_model sized activation and output linear mapping to assign probability to each of the vocab
  
  def forward(self, x):
    return F.log_softmax(self.proj(x), dim=-1)

# Define Model


In [ ]:
class Model(nn.Module):
  def __init__(self, encoder_decoder, generator):
    super().__init__()
    self.encoder_decoder = encoder_decoder
    self.generator = generator

  def forward(self, src, trg_x, src_key_padding_mask, tgt_mask, tgt_key_padding_mask):
    out = self.encoder_decoder.forward(src, trg_x, tgt_mask = tgt_mask)
    out = self.encoder_decoder.forward(src, trg_x, src_key_padding_mask = src_key_padding_mask, tgt_mask = tgt_mask, tgt_key_padding_mask =  tgt_key_padding_mask)
    out = self.generator.forward(out)
    return out

In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [ ]:
import functools, traceback, sys
def gpu_mem_restore(func):
    "Reclaim GPU RAM if CUDA out of memory happened, or execution was interrupted"
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except:
            type, val, tb = sys.exc_info()
            traceback.clear_frames(tb)
            raise type(val).with_traceback(tb) from None
    return wrapper

In [ ]:
def forward_pass(batch, model):
  src = batch.src.to(device=device)
  trg_x = batch.trg_x.to(device=device)
  src_padding_mask = batch.src_padding_mask.to(device=device)
  trg_attn_mask = batch.trg_attn_mask.to(device=device)
  trg_padding_mask = batch.trg_padding_mask.to(device=device)
  pred = model.forward(src, trg_x, src_padding_mask, trg_attn_mask, trg_padding_mask).view(-1, len(de_vocab.stoi))
  return pred
  
def calculate_loss(batch, prediction):
  trg_y = batch.trg_y.to(device=device).reshape(-1)
  loss = criterion(prediction, trg_y)  
  return loss

@gpu_mem_restore
def train_epoch(data_iterator, model):
  model.train() 
  total_loss = 0
  total_tokens = 0
  for i, batch in enumerate(data_iterator):
    # print("------ batch ", i, ", memory allocated: ", torch.cuda.memory_allocated() / 1e9, "-----")
    # forward pass
    pred = forward_pass(batch, model)
    loss = calculate_loss(batch, pred)
    # log info
    total_loss += float(loss)
    total_tokens += batch.ntokens
    if i % 50 == 1:
      print("Current average train loss: ", total_loss / total_tokens)
    # backward pass
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    # save model periodicaly
    if i % 1000 == 1:
      torch.save(model.state_dict(), path)
      china_now = datetime.datetime.now(tz)
      print("Saved model at: ", china_now, " memory left: ", torch.cuda.memory_allocated() / 1e9)
  scheduler.step()
  print("Average train loss for this epoch is: ", total_loss / total_tokens)

@gpu_mem_restore
def evaluate_epoch(data_iterator, model):
  model.eval()
  total_loss = 0
  total_tokens = 0
  with torch.no_grad():
    for i, batch in enumerate(data_iterator):
      # forward pass
      pred = forward_pass(batch, model)
      loss = calculate_loss(batch, pred)  
      # log info
      total_loss += float(loss)
      total_tokens += batch.ntokens
  print("Average valid loss for this epoch is: ", total_loss / total_tokens)

In [ ]:
# variables
pad_value = de_vocab.stoi['<pad>']; print("## pad_value: ", pad_value)
tgt_vocab_size = len(de_vocab.itos)

# initialize model
encoder_decoder = nn.Transformer(d_model = d_model, num_encoder_layers = 3, num_decoder_layers = 3, dim_feedforward = 512)
generator = Generator(d_model, tgt_vocab_size)
model = Model(encoder_decoder, generator).to(device)
model.apply(initialize_weights)

# hyperparameters
lr = 0.05
criterion = nn.CrossEntropyLoss(ignore_index = pad_value)
optimizer = torch.optim.Adam(model.encoder_decoder.parameters(), lr=lr) 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

## pad_value:  1


In [ ]:
# Run these lines a few times to free memory if you encounter leakage caused by exceptions and such
gc.collect()
torch.cuda.empty_cache()
type, val, tb = sys.exc_info()
traceback.clear_frames(tb)
assert False, "please liberate my GPU!"
# Reference: https://fastai1.fast.ai/troubleshoot.html

AssertionError: ignored

In [ ]:
# check memory allocated in GB
torch.cuda.memory_allocated() / 1e9

0.074151936

In [ ]:
def get_cudas():
    '''Returns the number of tensors in cuda device.'''
    n = 0
    for o in gc.get_objects():
        if torch.is_tensor(o):
            if o.is_cuda: 
              n += 1
    return n
get_cudas()

/usr/local/lib/python3.6/dist-packages/torch/distributed/distributed_c10d.py:144: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


96

# Start Training

In [ ]:
# encoder / decoder layer == 3, hidden dimension 256, heads 8
for epoch in range(10):
  print("------------Training epoch ", epoch, ", memory allocated: ", torch.cuda.memory_allocated() / 1e9, "--------------")
  train_epoch(train_dataloader, model)
  print("------------Evaluating epoch ", epoch, ", memory allocated: ", torch.cuda.memory_allocated() / 1e9, "--------------")
  evaluate_epoch(valid_dataloader, model)

------------Training epoch  0 , memory allocated:  0.074151936 --------------
Current average train loss:  0.036877972952624474
Saved model at:  2021-02-11 01:03:49.745539+08:00
Current average train loss:  0.03991694969425533
Current average train loss:  0.03905484801269985
Current average train loss:  0.03889268397230646
Current average train loss:  0.03912892083267474
Current average train loss:  0.038750869505649085
Current average train loss:  0.038630116347020425
Current average train loss:  0.038489317789017194
Current average train loss:  0.038298179783775296
Current average train loss:  0.038241517314019126
Current average train loss:  0.038195833151498604
Current average train loss:  0.03808395982741019
Current average train loss:  0.03795680543160932
Current average train loss:  0.03788552991154166
Current average train loss:  0.03778556403434982
Current average train loss:  0.03765741845980408
Current average train loss:  0.0376193145520135
Current average train loss:  0.03

In [ ]:
# load the model
# state_dict = torch.load(path)
# model.load_state_dict(state_dict)